In [1]:
import spiceypy as spice
import numpy as np
import pandas as pd
import pdr
import pvl

In [24]:
odf_file = "../data/odf/3004003a.odf"


In [25]:
odf_data = pdr.read(odf_file)
print(f'keys: {odf_data.keys()}')
odf_data

keys: ['LABEL', 'ODF1A_TABLE', 'ODF1B_TABLE', 'ODF2A_TABLE', 'ODF2B_TABLE', 'ODF3A_TABLE', 'ODF3B_TABLE', 'ODF4A64_TABLE', 'ODF4B64_TABLE', 'ODF4A43_TABLE', 'ODF4B43_TABLE', 'ODF4A63_TABLE', 'ODF4B63_TABLE', 'ODF4A14_TABLE', 'ODF4B14_TABLE', 'ODF4A61_TABLE', 'ODF4B61_TABLE', 'ODF4A15_TABLE', 'ODF4B15_TABLE', 'ODF4A45_TABLE', 'ODF4B45_TABLE', 'ODF4A65_TABLE', 'ODF4B65_TABLE', 'ODF4A12_TABLE', 'ODF4B12_TABLE', 'ODF4A42_TABLE', 'ODF4B42_TABLE', 'ODF8A_TABLE', 'ODF8B_TABLE']


pdr.Data(/home/sun/PycharmProjects/odf_parse/Maggelan-Orbit/src/../data/odf/3004003a.odf)
keys=['LABEL', 'ODF1A_TABLE', 'ODF1B_TABLE', 'ODF2A_TABLE', 'ODF2B_TABLE', 'ODF3A_TABLE', 'ODF3B_TABLE', 'ODF4A64_TABLE', 'ODF4B64_TABLE', 'ODF4A43_TABLE', 'ODF4B43_TABLE', 'ODF4A63_TABLE', 'ODF4B63_TABLE', 'ODF4A14_TABLE', 'ODF4B14_TABLE', 'ODF4A61_TABLE', 'ODF4B61_TABLE', 'ODF4A15_TABLE', 'ODF4B15_TABLE', 'ODF4A45_TABLE', 'ODF4B45_TABLE', 'ODF4A65_TABLE', 'ODF4B65_TABLE', 'ODF4A12_TABLE', 'ODF4B12_TABLE', 'ODF4A42_TABLE', 'ODF4B42_TABLE', 'ODF8A_TABLE', 'ODF8B_TABLE']
not yet loaded: ('LABEL', 'ODF1A_TABLE', 'ODF1B_TABLE', 'ODF2A_TABLE', 'ODF2B_TABLE', 'ODF3A_TABLE', 'ODF3B_TABLE', 'ODF4A64_TABLE', 'ODF4B64_TABLE', 'ODF4A43_TABLE', 'ODF4B43_TABLE', 'ODF4A63_TABLE', 'ODF4B63_TABLE', 'ODF4A14_TABLE', 'ODF4B14_TABLE', 'ODF4A61_TABLE', 'ODF4B61_TABLE', 'ODF4A15_TABLE', 'ODF4B15_TABLE', 'ODF4A45_TABLE', 'ODF4B45_TABLE', 'ODF4A65_TABLE', 'ODF4B65_TABLE', 'ODF4A12_TABLE', 'ODF4B12_TABLE', 'ODF4A42_TABL

In [26]:
def print_table_data(odf_file):
    odf_data = pdr.read(odf_file)
    
    orbit_data_table = odf_data['ODF3B_TABLE']
    
    print(f"Available columns in ODF3B_TABLE: {orbit_data_table.columns.tolist()}")
    print(f"Number of records: {len(orbit_data_table)}")
    
print_table_data(odf_file)

Available columns in ODF3B_TABLE: ['TIME TAG - INTEGER PART', 'TIME TAG - FRACTIONAL PART', 'OBSERVABLE - INTEGER PART', 'OBSERVABLE - FRACTIONAL PART', 'ITEMS 5-19', 'FREQUENCY - PART 1', 'FREQUENCY - PART 2', 'ITEM 22']
Number of records: 1172664


In [51]:
def print_structure(odf_file):
    odf_data = pdr.read(odf_file)
    orbit_data_table = odf_data['ODF3B_TABLE']
    print("\nITEMS 5-19 structure:")
    item = orbit_data_table['ITEMS 5-19'][10]
    print(f"ITEMS 5-19 = {item}")
    print(f"  Length: {len(item)} elements")
    for j, item_val in enumerate(item):
        print(f"    Item {j}: '{item_val}' (length: {len(item_val)})")

print_structure(odf_file)


ITEMS 5-19 structure:
ITEMS 5-19 = ['111', '1101000', '0000000', '00', '00', '000000', '0000', '00101000', '0010110000', '10', '1111100', '01', '00010000000', '0', '000000000000000000000000']
  Length: 15 elements
    Item 0: '111' (length: 3)
    Item 1: '1101000' (length: 7)
    Item 2: '0000000' (length: 7)
    Item 3: '00' (length: 2)
    Item 4: '00' (length: 2)
    Item 5: '000000' (length: 6)
    Item 6: '0000' (length: 4)
    Item 7: '00101000' (length: 8)
    Item 8: '0010110000' (length: 10)
    Item 9: '10' (length: 2)
    Item 10: '1111100' (length: 7)
    Item 11: '01' (length: 2)
    Item 12: '00010000000' (length: 11)
    Item 13: '0' (length: 1)
    Item 14: '000000000000000000000000' (length: 24)


In [28]:
from datetime import datetime, timedelta

def parse_binary_string(bin_str):
    return int(bin_str, 2)
    
def parse_odf(odf_file):
    odf_data = pdr.read(odf_file)

    orbit_data_table = odf_data['ODF3B_TABLE']
    
    results = []
    
    # spice_epoch = datetime(1950, 1, 1)
    
    for idx, record in orbit_data_table.iterrows():
        try:
            time_integer = record['TIME TAG - INTEGER PART']
            time_fractional = record['TIME TAG - FRACTIONAL PART']
            
            full_time_seconds = time_integer + time_fractional * 1e-9
            # 
            # record_time = spice_epoch + timedelta(seconds=full_time_seconds)
            
            items_5_19 = record['ITEMS 5-19']
            
            # Согласно выводу, структура ITEMS 5-19:
            # ['010', '0001111', '0001111', '01', '00', '110010', '1010', '00000000', '0000010010', '10', '0000000', '00', '01001110001', '1', '000101001011110110110100']
            # Индексы:
            # 0 - 5: Format ID (3 бита)
            # 1 - 6: Receiving Station ID (7 бит)
            # 2 - 7: Transmitting Station ID (7 бит)
            # 3 - 8: Network ID (2 бита)
            # 4 - 9: Downlink Band ID (2 бита)
            # 5 - 10: Data Type ID (6 бит)
            # 6 - 11: Highest Ranging Component (4 бита)
            # 7 - 12: Spacecraft/Quasar flag + Spacecraft ID (8 бит)
            # 8 - 13: Pass ID / Quasar ID (10 бит)
            # 9 - 14: Split Pass ID / Modulus indicator (2 бита)
            # 10 - 15: 2nd Receiving Station ID (7 бит)
            # 11 - 16: Uplink Band ID (2 бита)
            # 12 - 17: Power/Noise Ratio + Modulus integer (7+7=14 бит?)
            # 13 - 18: Data Validity (1 бит)
            # 14 - 19: Compression Time / Ranging / Modulus fractional (24 бита)
            # print(items_5_19)
            if len(items_5_19) >= 6:
                format_id = parse_binary_string(items_5_19[0])  # биты 129-131
                station_id = parse_binary_string(items_5_19[1])  # биты 132-138
                data_type_id = parse_binary_string(items_5_19[5])  # биты 150-155
                network_id = parse_binary_string(items_5_19[3])
                
                # One-way Doppler имеет Data Type ID = 11
                if data_type_id == 11 and format_id == 1:
                    # Observable (Doppler) - слова 3 и 4
                    doppler_integer = record['OBSERVABLE - INTEGER PART']
                    doppler_fractional = record['OBSERVABLE - FRACTIONAL PART']
                    #compression_time = parse_binary_string(items_5_19[14])
                    
                    # Полное значение допплера 
                    one_way_doppler = doppler_integer + (doppler_fractional * 1e-9) #+ (compression_time * 1e-7)
                    # # Residual - из слова 9, биты 265-288 (ITEM 22)
                    # residual = record.get('ITEM 22', 0) * 1e-3  # умножаем на 10^-3
                    
                    transmitting_station_id = parse_binary_string(items_5_19[2])
                    
                    results.append({
                        't1': time_integer,
                        't2': time_fractional,
                        't3': full_time_seconds,
                        # 't3': record_time,
                        'station_id': station_id,
                        'one_way_doppler': one_way_doppler,
                        # 'residual': residual,
                        'format_id': format_id,
                        'transmitting_station_id': transmitting_station_id,
                    })
                
        except Exception as e:
            if idx < 5:  # Печатаем ошибки только для первых 5 записей
                print(f"Error processing record {idx}: {e}")
            continue
    
    return pd.DataFrame(results)


df = parse_odf(odf_file)

# print_structure(odf_file)
print(f"Found {len(df)} one-way Doppler records")

if len(df) > 0:
    print("\nFirst 10 records:")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(df.head(10))

    csv_file = "../data/csv/3004003a.csv"
    df.to_csv(csv_file, index=False)
    print(f"\nData saved to {csv_file}")

    # Дополнительная статистика
    print(f"\nStatistics:")
    print(f"Time range: {df['t3'].min()} to {df['t3'].max()}")
    print(f"Stations: {sorted(df['station_id'].unique())}")
    print(f"Doppler range: {df['one_way_doppler'].min():.6f} to {df['one_way_doppler'].max():.6f} Hz")

else:
    print("No one-way Doppler data found in the ODF file")



Found 501 one-way Doppler records

First 10 records:
      t1     t2            t3  station_id  one_way_doppler  format_id  \
0  64000  20711  64000.000021          66     1.767440e+09          1   
1  64000  20727  64000.000021          44     1.312227e+09          1   
2  64000  20729  64000.000021          87    -6.163661e+08          1   
3  64000  20733  64000.000021          12    -4.455793e+08          1   
4  64000  20734  64000.000021          83     1.352729e+09          1   
5  64000  20736  64000.000021          74    -8.512471e+08          1   
6  64000  20736  64000.000021          61    -3.438674e+08          1   
7  64000  20739  64000.000021          79     5.682627e+08          1   
8  64000  20740  64000.000021         125     1.544225e+09          1   
9  64000  20740  64000.000021          91    -1.949762e+09          1   

   transmitting_station_id  
0                       11  
1                       83  
2                       23  
3                      101 

In [29]:
def check_file_metadata(odf_file):
    """Проверяет метаданные файла"""
    odf_data = pdr.read(odf_file)
    
    print("\n=== МЕТАДАННЫЕ ФАЙЛА ===")
    
    # Проверяем все таблицы
    for key in odf_data.keys():
        if 'TABLE' in key:
            print(f"\n{key}:")
            table = odf_data[key]
            print(f"  Размер: {table.shape}")
            if hasattr(table, 'columns'):
                print(f"  Колонки: {table.columns.tolist()}")
            
            # Смотрим на первые значения если это не ODF3B
            if '3B' not in key and len(table) > 0:
                print(f"  Первая запись: {table.iloc[0].to_dict()}")

# Проверяем метаданные
check_file_metadata(odf_file)



=== МЕТАДАННЫЕ ФАЙЛА ===

ODF1A_TABLE:
  Размер: (1, 4)
  Колонки: ['PRIMARY KEY', 'SECONDARY KEY', 'LOGICAL RECORD LENGTH', 'GROUP START PACKET NUMBER']
  Первая запись: {'PRIMARY KEY': 101, 'SECONDARY KEY': 0, 'LOGICAL RECORD LENGTH': 1, 'GROUP START PACKET NUMBER': 0}

ODF1B_TABLE:
  Размер: (1, 7)
  Колонки: ['SYSTEM ID', 'PROGRAM ID', 'SPACECRAFT ID', 'FILE CREATION DATE', 'FILE CREATION TIME', 'FILE REFERENCE DATE', 'FILE REFERENCE TIME']
  Первая запись: {'SYSTEM ID': b'rdca    ', 'PROGRAM ID': b'rkntdf2o', 'SPACECRAFT ID': 18, 'FILE CREATION DATE': 101027, 'FILE CREATION TIME': 191911, 'FILE REFERENCE DATE': 19500101, 'FILE REFERENCE TIME': 0}

ODF2A_TABLE:
  Размер: (1, 4)
  Колонки: ['PRIMARY KEY', 'SECONDARY KEY', 'LOGICAL RECORD LENGTH', 'GROUP START PACKET NUMBER']
  Первая запись: {'PRIMARY KEY': 107, 'SECONDARY KEY': 0, 'LOGICAL RECORD LENGTH': 1, 'GROUP START PACKET NUMBER': 2}

ODF2B_TABLE:
  Размер: (1, 3)
  Колонки: ['ITEM 1', 'ITEM 2', 'ITEM 3']
  Первая запись: {'